# André Luis Lessa Junior - RNN com Biblioteca

Base de dados:
- https://www.kaggle.com/datasets/gpreda/covid-world-vaccination-progress

Tutorial utilizado:

- https://www.geeksforgeeks.org/time-series-forecasting-using-recurrent-neural-networks-rnn-in-tensorflow/


# Setup

Instalar dependências, pacotes e upload da base de dados.

In [84]:
# Importar bibliotecas necessárias
import pandas as pd
import math
import numpy as np
import tensorflow as tf
from google.colab import files
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import GRU, Bidirectional
from keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [27]:
# Instalar dependências necessárias
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d gpreda/covid-world-vaccination-progress
!unzip covid-world-vaccination-progress.zip

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/gpreda/covid-world-vaccination-progress
License(s): CC0-1.0
covid-world-vaccination-progress.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  covid-world-vaccination-progress.zip
replace country_vaccinations.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: country_vaccinations.csv  
replace country_vaccinations_by_manufacturer.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: country_vaccinations_by_manufacturer.csv  


In [62]:
# Importar base de dados
base = pd.read_csv('country_vaccinations.csv')

# Analise Exploratória

Este dataset obtém informações de diversos países com relação a vacinação da COVID, mostrando também quais vacinas estavam sendo utilizadas e qual a fonte dos dados.

In [63]:
base.head(5)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


## Colunas

* Country - País em que a vacina está sendo aplicada;

* Country ISO Code - Código do país;

* Date - Data em que os dados foram atualizados;

* Total number of vaccinations - Valor absoluto da vacinação por país;

* Total number of people vaccinated - Número de pessoas que foram aplicadas vacinas (Existem pessoas que tomou duas doses, então o número de pessoas será maior que o número de pessoas no país);

* Total number of people fully vaccinated - Número de pessoas que foram totalmente vacinadas de acordo com o plano de imunização do país, geralmente é 2 vacinas;

* Daily vaccinations (raw) - Para uma determinada data, o número de vacinas aplicadas nesse dia/país;

* Daily vaccinations - Para uma determinada data, o número de vacinas aplicadas nesse dia/país;

* Total vaccinations per hundred - número de pessoas que foram vacinadas dividas pela quantidade de pessoas por país;

* Total number of people fully vaccinated per hundred - número de pessoas que foram completamente vacinadas dividia pela quantidade de pessoas por país;

* Daily vaccinations per million - divisão entre o número de pessoas vacinadas e a quantidade de pessoas naquele país atualizado;

* Vaccines used in the country - Número de vacinas usadas naquele país atualizado;

* Source name - Fonte em que os dados foram resgatados;

* Source website - Site em que os dados foram resgatados;

## Filtro

Nessa parte irei filtrar apenas as colunas que são importantes para termos a variação temporal e vamos tentar prever de acordo com as vacinas diárias quantas vacinas serão aplicadas, entendo que isso depende de diversos fatores como logística, preço das vacinas e plano de governo de cada país, porém essa ativiade é apenas para colocar em práticas os conceitos utilizados em sala de aula.

In [64]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86512 entries, 0 to 86511
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              86512 non-null  object 
 1   iso_code                             86512 non-null  object 
 2   date                                 86512 non-null  object 
 3   total_vaccinations                   43607 non-null  float64
 4   people_vaccinated                    41294 non-null  float64
 5   people_fully_vaccinated              38802 non-null  float64
 6   daily_vaccinations_raw               35362 non-null  float64
 7   daily_vaccinations                   86213 non-null  float64
 8   total_vaccinations_per_hundred       43607 non-null  float64
 9   people_vaccinated_per_hundred        41294 non-null  float64
 10  people_fully_vaccinated_per_hundred  38802 non-null  float64
 11  daily_vaccinations_per_milli

Selecionei as colunas de datas para ver a variação temporal, país para considerar apenas o Brasil na qual irei ver a variação temporal, o número de vacinas aplicadas diariamente, que é a série na qual irei acompanhar, e como feature adicional que pode ajudar o modelo o número de vacinação por milhão.

In [65]:
base = base[['date','country','daily_vaccinations', 'daily_vaccinations_per_million']]

In [66]:
# Remover valores nulos na qual irá atrapalhar o acompanhamento
base = base.dropna()

In [67]:
# Filtrar apenas o Brazil, na qual irá diminuir a base de dados
base = base[base['country']=='Brazil']

In [68]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 436 entries, 10650 to 11085
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   date                            436 non-null    object 
 1   country                         436 non-null    object 
 2   daily_vaccinations              436 non-null    float64
 3   daily_vaccinations_per_million  436 non-null    float64
dtypes: float64(2), object(2)
memory usage: 17.0+ KB


In [69]:
# Verificação se a data está no formato correto, ano, mês e data, pois é mais fácil de identificar a sequência
base['date'].head(1)

,date
10650,2021-01-18


In [70]:
# O período de vacinação começou em janeiro de 2021 e finalizou em março de 2022
print('\n', min(base['date']))
print('\n', max(base['date']))


 2021-01-18

 2022-03-29


In [71]:
# Tendo 436 dias registrados que é em torno de 1 ano 2 meses e 10 dias
len(base['date'].unique())

436

Primeiro será necessário ordernar os valores por data, pois as datas não são inseridas no modelo, isso significa que, os valores das vacinas diárias devem ser sequenciais, pois considerando a data t a linha seguinte deve ser t+1, assim os valores ficariam distribuidos.

In [72]:
base = base.sort_values(by='date')
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 436 entries, 10650 to 11085
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   date                            436 non-null    object 
 1   country                         436 non-null    object 
 2   daily_vaccinations              436 non-null    float64
 3   daily_vaccinations_per_million  436 non-null    float64
dtypes: float64(2), object(2)
memory usage: 17.0+ KB


## Gráfico

In [73]:
# prompt: chart date x daily_vaccinations

fig = px.line(base, x='date', y='daily_vaccinations', title='Variação das Vacinas Aplicadas no Brasil')
fig.update_layout(xaxis_title='Data', yaxis_title='Número de Vacinas Aplicadas por dia')
fig.show()


O comportamento do gráfico é normal, já que existem diversas variáveis que podem atrapalhar a vacinação diária como:

* Logística
* Disponibilidade de Vacinas
* Preço das vacinas
* Rejeição da população

Além disso percebe-se um pico em agosto e janeiro, enquanto um dos menores registros após 9 meses de vacinação também foi em janeiro

# Preparação dos dados

Nessa seção será realizada a preparação para inserir os dados no modelo, como separação de treino e teste e normalização

In [76]:
# Dividir em conjunto de treino e teste na lógica 80:20
base_treinamento = math.ceil(len(base) * 0.8)

base_treino = base[:base_treinamento].iloc[:,2]
base_teste = base[base_treinamento:].iloc[:,:3]

print(base_treino.shape)
print(base_teste.shape)

(345,)
(86, 3)


In [77]:
# Selecionar apenas os valores das vacinas diariamente
dados_treino = base_treino.values
dados_teste = base_teste.daily_vaccinations.values

# Reshape os dados para apenas 1 dimensão
dados_treino = np.reshape(dados_treino, (-1,1))
dados_teste = np.reshape(dados_teste, (-1,1))

print(dados_treino.shape)
print(dados_teste.shape)

(345, 1)
(86, 1)


In [78]:
# Normalização dos dados entre 0 e 1
scaler = MinMaxScaler(feature_range=(0,1))
normalizado_treino = scaler.fit_transform(dados_treino)
normalizado_teste = scaler.fit_transform(dados_teste)
print(*normalizado_treino[:5])
print(*normalizado_teste[:5])

[0.] [0.0029434] [0.01764702] [0.02442958] [0.04712435]
[0.06965746] [0.02312329] [0.02067431] [0.00907663] [0.]


Para cada amostra de dados de treinamento, é criada uma sequência de 30 valores (X_train) e o próximo valor (y_train) é armazenado como o rótulo correspondente.

O mesmo processo é aplicado para os dados de teste, gerando sequências de 30 valores e o valor subsequente que a rede precisa prever.

In [82]:
X_train = []
y_train = []
for i in range(30, len(normalizado_treino)):
    X_train.append(normalizado_treino[i-30:i, 0])
    y_train.append(normalizado_treino[i, 0])
    if i <= 30:
        print(X_train)
        print(y_train)
        print()

X_test = []
y_test = []
for i in range(30, len(normalizado_teste)):
    X_test.append(normalizado_teste[i-30:i, 0])
    y_test.append(normalizado_teste[i, 0])

[array([0.        , 0.0029434 , 0.01764702, 0.02442958, 0.04712435,
       0.04871948, 0.0446819 , 0.05843617, 0.07548585, 0.08943854,
       0.10031125, 0.10709586, 0.10795812, 0.10773083, 0.10924917,
       0.10815706, 0.10805604, 0.11079689, 0.1082756 , 0.11045416,
       0.11150453, 0.11196374, 0.11718878, 0.11763614, 0.11885298,
       0.12640657, 0.12342039, 0.12380178, 0.1312595 , 0.12930668])]
[0.13200424064473468]

[array([0.        , 0.0029434 , 0.01764702, 0.02442958, 0.04712435,
       0.04871948, 0.0446819 , 0.05843617, 0.07548585, 0.08943854,
       0.10031125, 0.10709586, 0.10795812, 0.10773083, 0.10924917,
       0.10815706, 0.10805604, 0.11079689, 0.1082756 , 0.11045416,
       0.11150453, 0.11196374, 0.11718878, 0.11763614, 0.11885298,
       0.12640657, 0.12342039, 0.12380178, 0.1312595 , 0.12930668]), array([0.0029434 , 0.01764702, 0.02442958, 0.04712435, 0.04871948,
       0.0446819 , 0.05843617, 0.07548585, 0.08943854, 0.10031125,
       0.10709586, 0.10795812, 0.

In [83]:
# The data is converted to Numpy array
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

y_train = np.reshape(y_train, (y_train.shape[0],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))

print("X_train :",X_train.shape,"y_train :",y_train.shape)
print("X_test :",X_test.shape,"y_test :",y_test.shape)


X_train : (315, 30, 1) y_train : (315, 1)
X_test : (56, 30, 1) y_test : (56, 1)


# Construção da RNN

Nesta seção será construída a rede neural recorrente, apresentando qual métrica está sendo utilizada, arquitetura da rede e comparação entre valor predito e real.

A Rede Neural Recorrente (RNN) com três camadas SimpleRNN e uma camada densa de saída. A primeira camada RNN tem 30 neurônios, usa a ativação tanh, e inclui Dropout para evitar overfitting. As duas camadas intermediárias RNN também possuem 30 neurônios e mantêm o fluxo de sequências temporais para a próxima camada. A última camada densa usa a ativação sigmoid, produzindo uma saída final entre 0 e 1, adequada para previsões de séries temporais.

In [85]:
# Iniciando uma rede neural sequencial
model = Sequential()

# Adicionar camada de entrada com dropout
model.add(SimpleRNN(units = 30,
                        activation = "tanh",
                        return_sequences = True,
                        input_shape = (X_train.shape[1],1)))
model.add(Dropout(0.2))

# Adicionar camadas para a rede neural

model.add(SimpleRNN(units = 30,
                        activation = "tanh",
                        return_sequences = True))

model.add(SimpleRNN(units = 30,
                        activation = "tanh",
                        return_sequences = True))

model.add( SimpleRNN(units = 30))

# Adicionar o output da rede com uma sigmoid
model.add(Dense(units = 1,activation='sigmoid'))

# Sumário da rede
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 30, 30)              │             960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 30, 30)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 30, 30)              │           1,830 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 30, 30)              │           1,830 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_3 (SimpleRNN)             │ (None, 30)                  │           1,830 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,481 (25.32 KB)

 Trainable params: 6,481 (25.32 KB)

 Non-trainable params: 0 (0.00 B)

O SGD (Stochastic Gradient Descent) é um algoritmo de otimização que ajusta os pesos da rede neural com base no gradiente da função de perda em cada amostra, fazendo atualizações frequentes e rápidas. Com o uso de momentum (0.9) e Nesterov momentum, ele melhora a velocidade de convergência, evitando oscilações, especialmente em regiões mais complexas da função de erro. O decay reduz a taxa de aprendizado com o tempo, ajudando a estabilizar a convergência.

A função de perda usada é o Mean Squared Error (MSE), que calcula a média dos quadrados das diferenças entre os valores previstos e os valores reais, penalizando erros maiores de forma mais significativa. É uma métrica padrão para problemas de regressão, como a previsão de séries temporais.

Como métrica foi selecionada a mae (Mean Absolute Error), pois ela é usada em problemas de regressão para medir o erro absoluto médio das previsões. Ela calcula a média das diferenças absolutas entre os valores previstos e reais, oferecendo uma visão clara do erro médio do modelo em termos das unidades dos dados. É útil quando você deseja entender o erro médio sem penalizar erros maiores mais severamente, como no MSE, além disso já está sendo usada a MSE na função de perda, então para não ficar redondante será a MAE. É importante o uso dessas métricas, pois outras métricas como acurácia, f1-score, e etc servem para problemas de classificação e nesse caso é de regressão.









In [88]:
# Compilando o modelo
model.compile(optimizer = SGD(learning_rate=0.01,
                                  decay=1e-6,
                                  momentum=0.9,
                                  nesterov=True),
                  loss = "mean_squared_error",
               metrics=['accuracy','mae'])

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning:

Argument `decay` is no longer supported and will be ignored.



In [89]:
# Treinando o modelo
histoy = model.fit(X_train, y_train, epochs = 30, batch_size = 10)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.0016 - loss: 0.0500 - mae: 0.1746
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 6.0388e-04 - loss: 0.0107 - mae: 0.0791
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0023 - loss: 0.0085 - mae: 0.0713
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.0032 - loss: 0.0085 - mae: 0.0705
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.0038 - loss: 0.0065 - mae: 0.0608
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.0068 - loss: 0.0069 - mae: 0.0605
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0035 - loss: 0.0059 - mae: 0.0587
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 3.9116e-04 - loss: 0.0057 - mae: 0.0567
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0023 - loss: 0.0076 - mae: 0.0638
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0042 - loss: 0.0062 - mae: 0.0587
E

In [90]:
# Predições com os dados de teste
y_RNN = model.predict(X_test)
y_RNN_O = scaler.inverse_transform(y_RNN)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 478ms/step


# Resultados

A partir dos resultados percebe-se que o modelo está predizendo os 50 dias, o modelo não está com overfitting, ele começa errando com a loss e a mae bem ruins, minimizando de acordo com o treinamento.

In [103]:
# Extraindo dados das métricas
loss = histoy.history['loss']
mae = histoy.history['mae']

# Número de épocas (convertendo range para uma lista)
epochs = list(range(1, len(loss) + 1))


# Gráfico da perda (loss)
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(x=epochs, y=loss, mode='lines+markers', name='Perda (Loss)'))
fig_loss.update_layout(
    title="Evolução da Perda ao Longo das Épocas",
    xaxis_title="Épocas",
    yaxis_title="Perda (Loss)",
    width=800,
    height=500
)
fig_loss.show()


In [104]:
# Gráfico do erro absoluto médio (MAE)
fig_mae = go.Figure()
fig_mae.add_trace(go.Scatter(x=epochs, y=mae, mode='lines+markers', name='Erro Absoluto Médio (MAE)'))
fig_mae.update_layout(
    title="Evolução do Erro Absoluto Médio (MAE) ao Longo das Épocas",
    xaxis_title="Épocas",
    yaxis_title="Erro Absoluto Médio (MAE)",
    width=800,
    height=500
)
fig_mae.show()

Com a comparação percebe-se que o modelo não está tão assertivo, talvez seja necessário adicionar mais dias, é importante notar que apesar de todas as variáveis que influenciam a quantidade de vacinação no dia, o modelo de rede neural está conseguindo ficar bem próximo, porém há oportunidades de melhorias.

In [93]:
# Conjunto de dados reais (dados de teste reais)
dados_reais = scaler.inverse_transform(y_test)

# Criando uma figura com Plotly
fig = go.Figure()

# Adicionando os dados reais ao gráfico
fig.add_trace(go.Scatter(x=np.arange(len(dados_reais)), y=dados_reais.flatten(), mode='lines', name='Dados Reais'))

# Adicionando as predições ao gráfico
fig.add_trace(go.Scatter(x=np.arange(len(y_RNN_O)), y=y_RNN_O.flatten(), mode='lines', name='Predições RNN'))

# Personalizando o layout do gráfico
fig.update_layout(
    title="Comparação entre Dados Reais e Predições do Modelo RNN",
    xaxis_title="Dias",
    yaxis_title="Número de Vacinações Diárias",
    legend=dict(x=0, y=1, traceorder="normal"),
    width=800,
    height=500
)

# Exibindo o gráfico
fig.show()
